In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [3]:

# 

import spacy
import re
import pandas as pd
import numpy as np
import requests

import networkx as nx
import matplotlib.pyplot as plt

# from common import *

nlp = spacy.load('en_core_web_sm')
pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [326]:
def read_file(filename):
    with open(filename, encoding='utf-8', mode='r', errors='ignore') as file:
        return file.read().strip() 

    
def split_chapter(chapter):
    return re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', chapter.replace('\n', ' ').replace('  ', ' ').strip())

    
def get_one_chapter_strang(chapter_number, book, subsections=False):
    s_chapter_number = str(chapter_number)
    chapter_starts = re.search('Chapter\s' + s_chapter_number +'\s\n.*\s\n' + s_chapter_number + '\.1.*\n', book)
    chapter_ends = re.search('Chapter\s' + str(chapter_number + 1) + '\s\n.*\s\n' + str(chapter_number + 1) + '\.1', 
                             book[chapter_starts.start():])
    chapter = book[chapter_starts.start():chapter_starts.start()+chapter_ends.start()]

    # TODO: check if this gavnocode works
    
    subs_start = re.search(s_chapter_number + '\.[0-9]+\s.*', chapter)
    i = 1
    
    while True:
        subs_end = re.search('\nProblem\sSet\s' + s_chapter_number + '\.' + str(i) + '+\s', chapter)
        
        if not subs_end:
#             print(chapter.count('Problem Set'), chapter)
            break
    
        print(subs_end.start(), chapter[subs_end.start():subs_end.end()])
        print(subs_start.start(), chapter[subs_start.start():subs_start.end()])

        subs_name = chapter[subs_start.start():subs_start.end() - 1] # exclude \n

        subs = chapter[subs_start.end():subs_end.start()]

        if subsections:
            yield subs_name[subs_name.find(' ') + 1:], split_chapter(subs)
        
        chapter = chapter[subs_end.end():]

        subs_start = re.search('('+s_chapter_number + '\.' + str(i) + '\s[^0-9]* )', chapter)

        print(subs_start.start(), chapter[subs_start.start():subs_start.end()])
        print('\n\n\n')
        break

#         if chapter.count('Problem Set ') == 1:  # very clever ifka, only sverhrazums top 5 understand
#             break
        i+= 1





In [327]:
book = read_file('data/Strang-Linear Algebra.txt')
chapter = dict(get_one_chapter_strang(3, book, subsections=True))


16707 
Problem Set 3.1 
26 3.1 Orthogonal Vectors and Subspaces 
3500 3.1 Orthogonal Vectors and Subspaces 






In [328]:
chapter.keys()


dict_keys(['Orthogonal Vectors and Subspaces'])

In [329]:
chapter

{'Orthogonal Vectors and Subspaces': ['A basis is a set of independent vectors that span a space.',
  'Geometrically, it is a set of coordinate axes.',
  'A vector space is defined without those axes, but every time I think of the x-y plane or three-dimensional space or Rn, the axes are there.',
  'They are usually perpendicular! The coordinate axes that the imagination constructs are practically always orthogonal.',
  'In choosing a basis, we tend to choose an orthogonal basis.',
  'The idea of an orthogonal basis is one of the foundations of linear algebra.',
  'We need a basis to convert geometric constructions into algebraic calculations, and we need an orthogonal basis to make those calculations simple.',
  'A further specialization makes the basis just about optimal: The vectors should have length 1.',
  'For an orthonormal basis (orthogonal unit vectors), we will find 1. the length kxk of a vector; 2. the test xTy = 0 for perpendicular vectors; and 3. how to create perpendicular

In [319]:
chapter['Projections and Least Squares']

['191 3.',
 'Solve Ax = b by least squares, and find p = Abx if A = 2 64 1 0 0 1 1 1 3 75 ; b = 2 64 1 1 0 3 75 : Verify that the error b. p is perpendicular to the columns of A. 4.',
 'Write out E2 = kAx.bk2 and set to zero its derivatives with respect to u and v, if A = 2 64 1 0 0 1 1 1 3 75 ; x = " u v # ; b = 2 64 1 3 4 3 75 : Compare the resulting equations with ATAbx = ATb, confirming that calculus as well as geometry gives the normal equations.',
 'Find the solution bx and the projection p = Abx.',
 'Why is p = b? 5.',
 'The following system has no solution: Ax = 2 64 1 .1 1 0 1 1 3 75 " C D # = 2 64 4 5 9 3 75 = b: Sketch and solve a straight-line fit that leads to the minimization of the quadratic (C.D.4)2+(C.5)2+(C+D.9)2?',
 'What is the projection of b onto the column space of A? 6.',
 'Find the projection of b onto the column space of A: A = 2 64 1 1 1 .1 .2 4 3 75 ; b = 2 64 1 2 7 3 75 : Split b into p+q, with p in the column space and q perpendicular to that space.',
 'Wh

In [253]:
chapter[' Introduction']

KeyError: ' Introduction'

In [254]:
list(get_one_chapter_strang(3, book, subsections=False))

26 3.1 Orthogonal Vectors and Subspaces 
16707 
Problem Set 3.1 
0 3.1 Orthogonal Vectors and Subspaces 
16968 
Problem Set 3.2 
0 3.2 Cosines and Projections onto Lines 
24073 
Problem Set 3.3 
0 3.3 Projections and Least Squares 
37141 
Problem Set 3.4 
0 3.4 Orthogonal Bases and Gram-Schmidt 
22087 
Problem Set 3.5 


[]

In [330]:
chapter = {'Orthogonal Vectors and Subspaces': ['A basis is a set of independent vectors that span a space.',
  'Geometrically, it is a set of coordinate axes.',
  'A vector space is defined without those axes, but every time I think of the x-y plane or three-dimensional space or Rn, the axes are there.',
  'They are usually perpendicular! The coordinate axes that the imagination constructs are practically always orthogonal.',
  'In choosing a basis, we tend to choose an orthogonal basis.',
  'The idea of an orthogonal basis is one of the foundations of linear algebra.',
  'We need a basis to convert geometric constructions into algebraic calculations, and we need an orthogonal basis to make those calculations simple.',
  'A further specialization makes the basis just about optimal: The vectors should have length 1.',
  'For an orthonormal basis (orthogonal unit vectors), we will find 1. the length kxk of a vector; 2. the test xTy = 0 for perpendicular vectors; and 3. how to create perpendicular vectors from linearly independent vectors.',
  'More than just vectors, subspaces can also be perpendicular.',
  'We will discover, so beautifully and simply that it will be a delight to see, that the fundamental subspaces meet at right angles.',
  'Those four subspaces are perpendicular in pairs, two in Rm and two in Rn. That will complete the fundamental theorem of linear algebra.',
  'The first step is to find the length of a vector.',
  'It is denoted by kxk, and in two dimensions it comes from the hypotenuse of a right triangle (Figure 3.1a).',
  'The square of the length was given a long time ago by Pythagoras: kxk2 = x2 1+x2 2.',
  'In three-dimensional space, x = (x1;x2;x3) is the diagonal of a box (Figure 3.1b).',
  'Its length comes from two applications of the Pythagorean formula.',
  'The two-dimensional case takes care of (x1;x2;0) = (1;2;0) across the base.',
  'This forms a right angle with the vertical side (0;0;x3)= (0;0;3).',
  'The hypotenuse of the bold triangle (Pythagoras again)\x0c 160 Chapter 3 Orthogonality b 1 2 p5 (1, 0) (0, 2) (1, 2) kxk 2 = x2 1 + x2 2 + x2 3 5 = 12 + 22 14 = 12 + 22 + 32 (a) (b) x (0, 0, 3) (1, 2, 3) has length p 14 (1, 0, 0) (0, 2, 0) (1, 2, 0) has length p 5 Figure 3.1: The length of vectors (x1;x2) and (x1;x2;x3). is the length kxk we want: Length in 3D kxk2 = 12+22+32 and kxk = q x2 1+x2 2+x2 3: The extension to x = (x1; : : : ;xn) in n dimensions is immediate.',
  'By Pythagoras n.1 times, the length kxk in Rn is the positive square root of xTx: Length squared kxk2 = x2 1+x2 2+. . .+x2 n = xTx: (1) The sum of squares matches xTxand the length of x = (1;2;.3) is p 14: xTx = h 1 2 .3 i 2 64 1 2 .3 3 75 = 12+22+(.3)2 = 14: Orthogonal Vectors How can we decide whether two vectors x and y are perpendicular?',
  'What is the test for orthogonality in Figure 3.2?',
  'In the plane spanned by x and y, those vectors are orthogonal provided they form a right triangle.',
  'We go back to a2+b2 = c2: Sides of a right triangle kxk2+kyk2 = kx.yk2: (2) Applying the length formula (1), this test for orthogonality in Rn becomes . x2 1+. . .+x2 n . + . y21 +. . .+y2n . = (x1.y1)2+. . .+(xn.yn)2: The right-hand side has an extra .2xiyi from each (xi.yi)2: right-hand side = . x2 1+. . .+x2 n . .2(x1y1+. . .+xnyn)+ . y21 +. . .+y2n . :\x0c 3.1 Orthogonal Vectors and Subspaces 161 y = \x14.1 2 \x15 x = \x144 2 \x15 p 25 p 20 p 5 xTy = 0 b Right angle xTy = 0 greater than 90 xTy < 0 less than 90 xTy > 0 Figure 3.2: A right triangle with 5+20 = 25.',
  'Dotted angle 100, dashed angle 30.',
  'We have a right triangle when that sum of cross-product terms xiyi is zero: Orthogonal vectors xTy = x1y1+. . .+xnyn = 0: (3) This sum is xTy = axiyi = yTx, the row vector xT times the column vector y: Inner product xTy = h x1 . . . xn i 2 64 y1 ... yn 3 75 = x1y1+. . .+xnyn: (4) This number is sometimes called the scalar product or dot product, and denoted by (x;y) or x . y.',
  'We will use the name inner product and keep the notation xTy. 3A The inner product xTy is zero if and only if x and y are orthogonal vectors.',
  'If xTy > 0, their angle is less than 90.',
  'If xTy < 0, their angle is greater than 90.',
  'The length squared is the inner product of x with itself: xTx = x2 1+. . .+x2 n = kxk2.',
  'The only vector with length zerothe only vector orthogonal to itselfis the zero vector.',
  'This vector x = 0 is orthogonal to every vector in Rn. Example 1. (2;2;.1) is orthogonal to (.1;2;2).',
  'Both have length p 4+4+1 = 3.',
  'Useful fact: If nonzero vectors v1; : : : ;vk are mutually orthogonal (every vector is perpendicular to every other), then those vectors are linearly independent.',
  'Proof.',
  'Suppose c1v1 +. . .+ckvk = 0.',
  'To show that c1 must be zero, take the inner product of both sides with v1.',
  'Orthogonality of the vs leaves only one term: vT1 (c1v1+. . .+ckvk) = c1vT1 v1 = 0: (5) The vectors are nonzero, so vT1 v1 6= 0 and therefore c1 = 0.',
  'The same is true of every ci.',
  'The only combination of the vs producing zero has all ci = 0: independence! The coordinate vectors e1; : : : ;en in Rn are the most important orthogonal vectors.',
  'Those are the columns of the identity matrix.',
  'They form the simplest basis for Rn, and\x0c 162 Chapter 3 Orthogonality they are unit vectorseach has length keik=1.',
  'They point along the coordinate axes.',
  'If these axes are rotated, the result is a new orthonormal basis: a new system of mutually orthogonal unit vectors.',
  'In R2 we have cos2q +sin2q = 1: Orthonormal vectors in R2 v1 = (cosq ; sinq ) and v2 = (.sinq ;cosq ): Orthogonal Subspaces We come to the orthogonality of two subspaces.',
  'Every vector in one subspace must be orthogonal to every vector in the other subspace.',
  'Subspaces of R3 can have dimension 0, 1, 2, or 3.',
  'The subspaces are represented by lines or planes through the origin and in the extreme cases, by the origin alone or the whole space.',
  'The subspace f0g is orthogonal to all subspaces.',
  'A line can be orthogonal to another line, or it can be orthogonal to a plane, but a plane cannot be orthogonal to a plane.',
  'I have to admit that the front wall and side wall of a room look like perpendicular planes in R3. But by our definition, that is not so! There are lines v and w in the front and side walls that do not meet at a right angle.',
  'The line along the corner is in both walls, and it is certainly not orthogonal to itself. 3B Two subspaces V and W of the same space Rn are orthogonal if every vector v in V is orthogonal to every vector w in W: vTw = 0 for all v and w.',
  'Example 2.',
  'Suppose V is the plane spanned by v1 = (1;0;0;0) and v2 = (1;1;0;0).',
  'If W is the line spanned by w = (0;0;4;5), then w is orthogonal to both vs.',
  'The line W will be orthogonal to the whole plane V.',
  'In this case, with subspaces of dimension 2 and 1 in R4, there is room for a third subspace.',
  'The line L through z = (0;0;5;.4) is perpendicular to V and W.',
  'Then the dimensions add to 2+1+1 = 4.',
  'What space is perpendicular to all of V, W, and L?',
  'The important orthogonal subspaces dont come by accident, and they come two at a time.',
  'In fact orthogonal subspaces are unavoidable: They are the fundamental subspaces! The first pair is the nullspace and row space.',
  'Those are subspaces of Rnthe rows have n components and so does the vector x in Ax = 0.',
  'We have to show, using Ax = 0, that the rows of A are orthogonal to the nullspace vector x. 3C Fundamental theorem of orthogonality The row space is orthogonal to the nullspace (in Rn).',
  'The column space is orthogonal to the left nullspace (in Rm).',
  'First Proof.',
  'Suppose x is a vector in the nullspace.',
  'Then Ax = 0, and this system of m\x0c 3.1 Orthogonal Vectors and Subspaces 163 equations can be written out as rows of A multiplying x: Every row is orthogonal to x Ax = 2 6664 . . . row 1 . . . . . . row 2 . . . ... ... ... . . . row m . . . 3 7775 2 6664 x1 x2 ... xn 3 7775 = 2 6664 0 0... 0 3 7775 : (6) The main point is already in the first equation: row 1 is orthogonal to x.',
  'Their inner product is zero; that is equation 1.',
  'Every right-hand side is zero, so x is orthogonal to every row.',
  'Therefore x is orthogonal to every combination of the rows.',
  'Each x in the nullspace is orthogonal to each vector in the row space, so N(A)?C(AT). The other pair of orthogonal subspaces comes from ATy = 0, or yTA = 0: yTA = h y1 . . . ym i 2 6666664 c c o o l l u . . . u m m n n 1 n 3 7777775 = h 0 . . . 0 i : (7) The vector y is orthogonal to every column.',
  'The equation says so, from the zeros on the right-hand side.',
  'Therefore y is orthogonal to every combination of the columns.',
  'It is orthogonal to the column space, and it is a typical vector in the left nullspace: N(AT)?C(A). This is the same as the first half of the theorem, with A replaced by AT. Second Proof.',
  'The contrast with this coordinate-free proof should be useful to the reader.',
  'It shows a more abstract method of reasoning.',
  'I wish I knew which proof is clearer, and more permanently understood.',
  'If x is in the nullspace then Ax = 0.',
  'If v is in the row space, it is a combination of the rows: v = ATz for some vector z.',
  'Now, in one line: Nullspace ?',
  'Row space vTx = (ATz)Tx = zTAx = zT0 = 0: (8) Example 3.',
  'Suppose A has rank 1, so its row space and column space are lines: Rank-1 matrix A = 2 64 1 3 2 6 3 9 3 75 : The rows are multiples of (1;3).',
  'The nullspace contains x=(.3;1), which is orthogonal to all the rows.',
  'The nullspace and row space are perpendicular lines in R2: h 1 3 i" 3 .1 # = 0 and h 2 6 i" 3 .1 # = 0 and h 3 9 i" 3 .1 # = 0:\x0c 164 Chapter 3 Orthogonality In contrast, the other two subspaces are in R3. The column space is the line through (1;2;3).',
  'The left nullspace must be the perpendicular plane y1 +2y2 +3y3 = 0.',
  'That equation is exactly the content of yTA = 0.',
  'The first two subspaces (the two lines) had dimensions 1+1 = 2 in the space R2. The second pair (line and plane) had dimensions 1+2 = 3 in the space R3. In general, the row space and nullspace have dimensions that add to r+(n.r) = n.',
  'The other pair adds to r+(m.r) = m.',
  'Something more than orthogonality is occurring, and I have to ask your patience about that one further point: the dimensions.',
  'It is certainly true that the null space is perpendicular to the row spacebut it is not the whole truth.',
  'N(A) contains every vector orthogonal to the row space.',
  'The nullspace was formed from all solutions to Ax = 0.',
  'Definition.',
  'Given a subspace V of Rn, the space of all vectors orthogonal to V is called the orthogonal complement of V.',
  'It is denoted by V? = V perp.',
  'Using this terminology, the nullspace is the orthogonal complement of the row space: N(A) = (C(AT))?.',
  'At the same time, the row space contains all vectors that are orthogonal to the nullspace.',
  'A vector z cant be orthogonal to the nullspace but outside the row space.',
  'Adding z as an extra row of A would enlarge the row space, but we know that there is a fixed formula r+(n.r) = n: Dimension formula dim(row space)+dim(nullspace) = number of columns.',
  'Every vector orthogonal to the nullspace is in the row space: C(AT) = (N(A))?.',
  'The same reasoning applied to AT produces the dual result: The left nullspace N(AT) and the column space C(A) are orthogonal complements.',
  'Their dimensions add up to (m.r)+r = m, This completes the second half of the fundamental theorem of linear algebra.',
  'The first half gave the dimensions of the four subspaces. including the fact that row rank = column rank.',
  'Now we know that those subspaces are perpendicular.',
  'More than that, the subspaces are orthogonal complements. 3D Fundamental Theorem of Linear Algebra, Part II The nullspace is the orthogonal complement of the row space in Rn. The left nullspace is the orthogonal complement of the column space in Rm. To repeat, the row space contains everything orthogonal to the nullspace.',
  'The column space contains everything orthogonal to the left nullspace.',
  'That is just a sentence, hidden in the middle of the book, but it decides exactly which equations can be solved! Looked at directly, Ax = b requires b to be in the column space.',
  'Looked at indirectly.',
  'Ax = b requires b to be perpendicular to the left nullspace. 3E Ax = b is solvable if and only if yTb = 0 whenever yTA = 0.\x0c 3.1 Orthogonal Vectors and Subspaces 165 The direct approach was b must be a combination of the columns.',
  'The indirect approach is b must be orthogonal to every vector that is orthogonal to the columns.',
  'That doesnt sound like an improvement (to put it mildly).',
  'But if only one or two vectors are orthogonal to the columns. it is much easier to check those one or two conditions yTb = 0.',
  'A good example is Kirchhoffs Voltage Law in Section 2.5.',
  'Testing for zero around loops is much easier than recognizing combinations of the columns.',
  'When the left-hand sides of Ax = b add to zero, the right-hand sides must, too: x1.x2 = b1 x2.x3 = b2 x3.x1 = b3 is solvable if and only if b1+b2+b3 = 0: Here A = 2 64 1 .1 0 0 1 .1 .1 0 1 3 75 : This test b1 +b2 +b3 = 0 makes b orthogonal to y = (1;1;1) in the left nullspace.',
  'By the Fundamental Theorem, b is a combination of the columns! The Matrix and the Subspaces We emphasize that V and W can be orthogonal without being complements.',
  'Their dimensions can be too small.',
  'The line V spanned by (0;1;0) is orthogonal to the line W spanned by (0;0;1), but V is not W?. The orthogonal complement of W is a twodimensional plane, and the line is only part of W?. When the dimensions are right, orthogonal subspaces are necessarily orthogonal complements: If W= V? then V =W? and dimV+dimW= n: In other words V?? = V.',
  'The dimensions of V and W are right, and the whole space Rn is being decomposed into two perpendicular parts (Figure 3.3).',
  'W V Two orthogonal axes in R3 Not orthogonal complements W V Line W perpendicular to plane V Orthogonal complements V =W?',
  'Figure 3.3: Orthogonal complements in R3: a plane and a line (not two lines).',
  'Splitting Rn into orthogonal parts will split every vector into x = v+w.',
  'The vector v is the projection onto the subspace V.',
  'The orthogonal component w is the projection of x onto W.',
  'The next sections show how to find those projections of x.',
  'They lead to what is probably the most important figure in the book (Figure 3.4).',
  'Figure 3.4 summarizes the fundamental theorem of linear algebra.',
  'It illustrates the true effect of a matrixwhat is happening inside the multiplication Ax. The nullspace\x0c 166 Chapter 3 Orthogonality Figure 3.4: The true action Ax = A(xrow+xnull) of any m by n matrix. is carried to the zero vector.',
  'Every Ax is in the column space.',
  'Nothing is carried to the left nullspace.',
  'The real action is between the row space and column space, and you see it by looking at a typical vector x.',
  'It has a row space component and a nullspace component, with x = xr+xn.',
  'When multiplied by A, this is Ax = Axr+Axn: The nullspace component goes to zero: Axn = 0.',
  'The row space component goes to the column space: Axr = Ax. Of course everything goes to the column spacethe matrix cannot do anything else.',
  'I tried to make the row and column spaces the same size, with equal dimension r. 3F From the row space to the column space, A is actually invertible.',
  'Every vector b in the column space comes from exactly one vector xr in the row space.',
  'Proof.',
  'Every b in the column space is a combination Ax of the columns.',
  'In fact, b is Axr, with xr in the row space, since the nullspace component gives Axn = 0, If another vector x0r in the row space gives Ax0r = b, then A(xr .x0r ) = b.b = 0.',
  'This puts xr .x0r in the nullspace and the row space, which makes it orthogonal to itself.',
  'Therefore it is zero, and xr.x0r .',
  'Exactly one vector in the row space is carried to b.',
  'Every matrix transforms its row space onto its column space.',
  'On those r-dimensional spaces A is invertible.',
  'On its nullspace A is zero.',
  'When A is diagonal, you see the invertible submatrix holding the r nonzeros.',
  'AT goes in the opposite direction, from Rm to Rn and from C(A) back to C(AT). Of course the transpose is not the inverse! AT moves the spaces correctly, but not the\x0c 3.1 Orthogonal Vectors and Subspaces 167 individual vectors.',
  'That honor belongs to A.1 if it existsand it only exists if r =m=n.',
  'We cannot ask A.1 to bring back a whole nullspace out of the zero vector.',
  'When A.1 fails to exist, the best substitute is the pseudoinverse A+. This inverts A where that is possible: A+Ax = x for x in the row space.',
  'On the left nullspace, nothing can be done: A+y = 0.',
  'Thus A+ inverts A where it is invertible, and has the same rank r.',
  'One formula for A+ depends on the singular value decompositionfor which we first need to know about eigenvalues.']}
